# 회귀 모델 평가지표: 캘리포니아 집값 예측을 통한 이해

이 노트북은 회귀 모델의 성능을 평가하는 데 사용되는 다양한 지표들을 다룹니다. `scikit-learn`의 캘리포니아 집값 데이터셋을 사용하여 선형 회귀 모델을 학습시키고, **평균 절대 오차(MAE), 평균 제곱 오차(MSE), 제곱근 평균 제곱 오차(RMSE), 결정 계수(R²)** 등 주요 평가지표들의 의미와 활용 방법을 상세히 설명합니다.

또한, 대규모 데이터셋의 시각화를 위한 샘플링 기법도 함께 다룹니다.

### 1. 라이브러리 임포트

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_california_housing 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler 


### 2. 데이터 로드 및 초기 탐색

`fetch_california_housing()` 함수를 사용하여 캘리포니아 집값 데이터셋을 불러옵니다. 이 데이터셋은 캘리포니아 지역의 인구 통계 데이터를 기반으로 주택 가격을 예측하는 데 사용됩니다.

In [ ]:
housing = fetch_california_housing()

# 특성(X)과 타겟(y)을 Pandas DataFrame/Series로 변환하여 다루기 쉽게 합니다.
X = pd.DataFrame(housing.data, columns=housing.feature_names)
y = pd.Series(housing.target, name="houseval") 

print("데이터 로드 성공!")
print("
특성 데이터 (X) 상위 5행:")
print(X.head())
print("
타겟 데이터 (y) 상위 10개:")
print(y[:10])
print("
데이터 정보:")
X.info()
print("
데이터 통계 요약:")
X.describe()

### 3. 데이터 분할

모델 학습과 평가를 위해 데이터를 훈련 세트와 테스트 세트로 분할합니다.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"훈련 데이터 형태: {X_train.shape}, {y_train.shape}")
print(f"테스트 데이터 형태: {X_test.shape}, {y_test.shape}")

### 4. 특성 스케일링

선형 회귀와 같은 모델은 특성들의 스케일에 민감할 수 있습니다. `StandardScaler`를 사용하여 특성들을 표준화합니다.

In [ ]:
scaler = StandardScaler() 
X_train_scaled = scaler.fit_transform(X_train) 
X_test_scaled = scaler.transform(X_test)  # 테스트 데이터는 훈련 데이터로 학습된 스케일러를 사용

print("스케일링 후 훈련 데이터 샘플 (일부):
", X_train_scaled[:2])

### 5. 모델 학습 및 예측

`LinearRegression` 모델을 학습시키고, 테스트 데이터에 대한 예측을 수행합니다.

In [ ]:
model = LinearRegression() 
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

print("예측된 값 샘플 (상위 10개):
", y_pred[:10])

### 6. 회귀 모델 평가지표

이제 예측된 값과 실제 값을 비교하여 모델의 성능을 다양한 지표로 평가합니다.

#### 6.1. 평균 절대 오차 (Mean Absolute Error, MAE)

예측 오차의 절대값 평균입니다. 오차의 크기를 직관적으로 파악할 수 있으며, 이상치에 덜 민감합니다.

`MAE = (1/n) * Σ |실제값 - 예측값|`

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE (평균 절대 오차): {mae:.4f}")

#### 6.2. 평균 제곱 오차 (Mean Squared Error, MSE)

예측 오차의 제곱 평균입니다. 오차가 큰 값에 더 큰 패널티를 부여하며, 딥러닝에서 손실 함수로 자주 사용됩니다. 이상치에 민감합니다.

`MSE = (1/n) * Σ (실제값 - 예측값)²`

In [ ]:
mse = mean_squared_error(y_test, y_pred)
print(f"MSE (평균 제곱 오차): {mse:.4f}")

#### 6.3. 제곱근 평균 제곱 오차 (Root Mean Squared Error, RMSE)

MSE에 제곱근을 취한 값입니다. MSE와 달리 원래 데이터와 단위가 같아 해석이 용이합니다.

`RMSE = √MSE`

In [ ]:
rmse = np.sqrt(mse)
print(f"RMSE (제곱근 평균 제곱 오차): {rmse:.4f}")

#### 6.4. 결정 계수 (R-squared, R²)

모델이 종속 변수의 분산을 얼마나 잘 설명하는지를 나타냅니다. 0과 1 사이의 값을 가지며, 1에 가까울수록 모델의 설명력이 높습니다. 음수 값이 나올 수도 있으며, 이는 모델이 평균으로 예측하는 것보다도 성능이 나쁘다는 의미입니다.

`R² = 1 - (모델의 오차 제곱합 / 실제값의 총 제곱합)`

**참고**: 특성의 개수가 많아지면 R² 값이 예측력과 상관없이 높아지는 경향이 있으므로 주의해야 합니다. 이 경우 MAE나 MSE를 함께 고려하는 것이 좋습니다.

In [ ]:
r2 = r2_score(y_test, y_pred) 
print(f"R^2 (결정 계수): {r2:.4f}")

# model.score() 함수도 R^2 값을 반환합니다.
print(f"모델의 score() 함수 결과 (R^2): {model.score(X_test_scaled, y_test):.4f}")

### 7. 데이터 샘플링 및 시각화

캘리포니아 집값 데이터셋은 약 2만 개의 샘플을 가지고 있어, 모든 특성 간의 관계를 `pairplot`으로 시각화하기에는 시간이 오래 걸리고 복잡합니다. 이럴 때는 데이터 샘플링을 통해 일부 데이터만 시각화하여 패턴을 파악할 수 있습니다.

In [ ]:
# 특성 데이터 X에 타겟 y를 추가하여 하나의 DataFrame으로 만듭니다.
df_combined = X.copy()
df_combined["houseval"] = y 

print(f"원본 결합 데이터 형태: {df_combined.shape}")

# 2000개의 샘플만 무작위로 추출합니다.
df_sample = df_combined.sample(n=2000, random_state=42) 
print(f"샘플링된 데이터 형태: {df_sample.shape}")

# pairplot 그리기
sns.pairplot(df_sample, 
             diag_kind='kde',  # 대각선에 커널 밀도 추정(KDE) 그래프 표시
             kind='scatter') # 산점도 그래프 표시
plt.suptitle('캘리포니아 집값 데이터셋 샘플링된 특성 관계', y=1.02) # 전체 제목
plt.show()

### 결론

회귀 모델의 성능을 평가할 때는 MAE, MSE, RMSE, R²와 같은 다양한 지표들을 종합적으로 고려해야 합니다. 각 지표는 모델의 다른 측면을 보여주므로, 문제의 특성과 목표에 따라 적절한 지표를 선택하고 해석하는 것이 중요합니다. 또한, 대규모 데이터셋의 탐색적 분석을 위해 데이터 샘플링과 시각화 기법을 활용하는 것도 효과적입니다.